In [23]:
DEBUG = False
LIVE = False # False = TEST, True = LIVE
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
ligands_smiles = ["CN1C=CC=C1", "COC(C1=CC=CO1)=O", "CCN(CC)C(=O)c1cc2ccccc2[nH]1"]
ts_guess_struct = "ts_guess.xyz"
bonds_to_remove=[(10, 41), (10, 12), (11, 41)]
n_cores = 10
mem_gb = 5
output_base_dir = "base_dir"
num_confs = 3
num_uff_conformers = 3
num_xtb_ff_conformers = 2
num_xtb_conformers_to_select = 1

In [25]:
# from frust.dirs import prepare_base_dir

# base_dir = prepare_base_dir(output_base_dir, ligands_smiles)
# print(base_dir)

In [ ]:
from frust.transformers import transformer_ts
from tooltoad.vis import MolTo3DGrid

ts_structs = {}
for smi in ligands_smiles:
    ts_test_mol = transformer_ts(
        ligand_smiles=smi,
        ts_guess_struct="../structures/ts1.xyz",
        pre_name="TS",
    )
    # extend each (mol, idxs) tuple with the smiles
    ts_with_smi = {
        name: (mol, idxs, smi)
        for name, (mol, idxs) in ts_test_mol.items()
    }
    ts_structs.update(ts_with_smi)
    
ts_structs

{'TS(1-methylpyrrole_rpos(2))': (<rdkit.Chem.rdchem.RWMol at 0x146ef0450>,
  [10, 11, 39, 40, 41, 44],
  'CN1C=CC=C1'),
 'TS(1-methylpyrrole_rpos(3))': (<rdkit.Chem.rdchem.RWMol at 0x146ef2bb0>,
  [10, 11, 39, 40, 41, 45],
  'CN1C=CC=C1'),
 'TS(methyl_furan-2-carboxylate_rpos(4))': (<rdkit.Chem.rdchem.RWMol at 0x14777cef0>,
  [10, 11, 39, 40, 41, 46],
  'COC(C1=CC=CO1)=O'),
 'TS(methyl_furan-2-carboxylate_rpos(5))': (<rdkit.Chem.rdchem.RWMol at 0x146e3a340>,
  [10, 11, 39, 40, 41, 47],
  'COC(C1=CC=CO1)=O'),
 'TS(methyl_furan-2-carboxylate_rpos(6))': (<rdkit.Chem.rdchem.RWMol at 0x105b7f2e0>,
  [10, 11, 39, 40, 41, 48],
  'COC(C1=CC=CO1)=O'),
 'TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(8))': (<rdkit.Chem.rdchem.RWMol at 0x14777fc90>,
  [10, 11, 39, 40, 41, 50],
  'CCN(CC)C(=O)c1cc2ccccc2[nH]1'),
 'TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(10))': (<rdkit.Chem.rdchem.RWMol at 0x146ef3150>,
  [10, 11, 39, 40, 41, 52],
  'CCN(CC)C(=O)c1cc2ccccc2[nH]1'),
 'TS(N,N-diethyl-1H-indole-2-car

In [27]:
from frust.embedder import embed_ts

mol, indices, smi = ts_structs.get("TS(methyl_furan-2-carboxylate_rpos(4))")
embedded_mol, conf_ids, atom_indices_to_keep, smi, energies = embed_ts(mol, indices, optimize=True)

Embedded 2 conformers on atom 46


In [28]:
embedded_dict1 = embed_ts(ts_structs, n_confs=5, optimize=False)
embedded_dict2 = embed_ts(ts_structs, n_confs=5, optimize=True)

Embedded 5 conformers on atom 44
Embedded 4 conformers on atom 45
Embedded 5 conformers on atom 46
Embedded 5 conformers on atom 47
Embedded 4 conformers on atom 48
Embedded 5 conformers on atom 50
Embedded 5 conformers on atom 52
Embedded 5 conformers on atom 53
Embedded 5 conformers on atom 54
Embedded 5 conformers on atom 55
Embedded 5 conformers on atom 44
Embedded 4 conformers on atom 45
Embedded 5 conformers on atom 46
Embedded 5 conformers on atom 47
Embedded 4 conformers on atom 48
Embedded 5 conformers on atom 50
Embedded 5 conformers on atom 52
Embedded 5 conformers on atom 53
Embedded 5 conformers on atom 54
Embedded 5 conformers on atom 55


In [29]:
from frust.stepper import Stepper
step = Stepper(
    debug = True,
    ligands_smiles=ligands_smiles
)

df1 = step.build_initial_df(embedded_dict1)
df2 = step.build_initial_df(embedded_dict2)

In [30]:
df1

,custom_name,ligand_name,rpos,constraint_atoms,cid,smiles,atoms,coords_embedded,energy_uff
0,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",0,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.2239454721163768, -1.5689961861050987, -0....",None
1,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",1,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-0.11186210392338937, 1.664496780604324, 0.5...",None
2,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",2,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-0.003487447036605559, 1.4525343475541515, -...",None
3,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",3,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-1.0196810753309713, 0.8421778342680636, -0....",None
4,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",4,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.5812412715997919, -1.4168195744206293, -0....",None
5,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",0,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.4390960857886995, -1.6328418133528142, -0....",None
6,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",1,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.9005569322225522, -1.571171609182764, 0.38...",None
7,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",2,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-1.130423699487623, -0.08462180729002457, 1....",None
8,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",3,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-1.3521654308051776, 1.1613032005858053, -0....",None
9,TS(methyl_furan-2-carboxylate_rpos(4)),methyl_furan-2-carboxylate,4,"[10, 11, 39, 40, 41, 46]",0,COC(C1=CC=CO1)=O,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-0.2293595587104927, -1.6934911667823793, -0...",None


In [31]:
from tooltoad.chemutils import ac2mol
loc = 40

atoms = df1["atoms"].iloc[loc]
coords_emb = df1["coords_embedded"].iloc[loc]
mol1 = ac2mol(atoms, coords_emb)

atoms = df2["atoms"].iloc[loc]
coords_opt = df2["coords_embedded"].iloc[loc]
mol2 = ac2mol(atoms, coords_emb)
MolTo3DGrid([mol1, mol2])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
step = Stepper(
    ligands_smiles=ligands_smiles,
    output_base="noob_dir",
    job_id=42,
    debug=True
)

In [33]:
df1 = step.build_initial_df(embedded_dict1)
df1

,custom_name,ligand_name,rpos,constraint_atoms,cid,smiles,atoms,coords_embedded,energy_uff
0,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",0,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.2239454721163768, -1.5689961861050987, -0....",None
1,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",1,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-0.11186210392338937, 1.664496780604324, 0.5...",None
2,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",2,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-0.003487447036605559, 1.4525343475541515, -...",None
3,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",3,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-1.0196810753309713, 0.8421778342680636, -0....",None
4,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",4,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.5812412715997919, -1.4168195744206293, -0....",None
5,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",0,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.4390960857886995, -1.6328418133528142, -0....",None
6,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",1,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.9005569322225522, -1.571171609182764, 0.38...",None
7,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",2,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-1.130423699487623, -0.08462180729002457, 1....",None
8,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",3,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-1.3521654308051776, 1.1613032005858053, -0....",None
9,TS(methyl_furan-2-carboxylate_rpos(4)),methyl_furan-2-carboxylate,4,"[10, 11, 39, 40, 41, 46]",0,COC(C1=CC=CO1)=O,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-0.2293595587104927, -1.6934911667823793, -0...",None


In [34]:
df1 = step.build_initial_df(embedded_dict1)
options = {
    "gfnff":        None,
    "ohess"  :        None,

}

detailed_inp_str = """
$write
   distances=true
   angles=true
   torsions=true
$end
"""

df = step.xtb(
    df=df1, options=options, detailed_inp_str=detailed_inp_str, constraint=True
)
df

[xtb-gfnff-ohess] row 0 (TS(1-methylpyrrole_rpos(2)))…
[xtb-gfnff-ohess] row 1 (TS(1-methylpyrrole_rpos(2)))…
[xtb-gfnff-ohess] row 2 (TS(1-methylpyrrole_rpos(2)))…
[xtb-gfnff-ohess] row 3 (TS(1-methylpyrrole_rpos(2)))…
[xtb-gfnff-ohess] row 4 (TS(1-methylpyrrole_rpos(2)))…
[xtb-gfnff-ohess] row 5 (TS(1-methylpyrrole_rpos(3)))…
[xtb-gfnff-ohess] row 6 (TS(1-methylpyrrole_rpos(3)))…
[xtb-gfnff-ohess] row 7 (TS(1-methylpyrrole_rpos(3)))…
[xtb-gfnff-ohess] row 8 (TS(1-methylpyrrole_rpos(3)))…
[xtb-gfnff-ohess] row 9 (TS(methyl_furan-2-carboxylate_rpos(4)))…
[xtb-gfnff-ohess] row 10 (TS(methyl_furan-2-carboxylate_rpos(4)))…
[xtb-gfnff-ohess] row 11 (TS(methyl_furan-2-carboxylate_rpos(4)))…
[xtb-gfnff-ohess] row 12 (TS(methyl_furan-2-carboxylate_rpos(4)))…
[xtb-gfnff-ohess] row 13 (TS(methyl_furan-2-carboxylate_rpos(4)))…
[xtb-gfnff-ohess] row 14 (TS(methyl_furan-2-carboxylate_rpos(5)))…
[xtb-gfnff-ohess] row 15 (TS(methyl_furan-2-carboxylate_rpos(5)))…
[xtb-gfnff-ohess] row 16 (TS(methyl_f

,custom_name,ligand_name,rpos,constraint_atoms,cid,smiles,atoms,coords_embedded,energy_uff,xtb-gfnff-ohess-opt_coords,xtb-gfnff-ohess-electronic_energy,xtb-gfnff-ohess-gibbs_energy,xtb-gfnff-ohess-vibs,xtb-gfnff-ohess-normal_termination
0,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",0,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.2239454721163768, -1.5689961861050987, -0....",None,"[[0.30109937249364, -1.64995020686268, -0.6682...",-8.678725,-7.997619,"[{'frequency': 16.89}, {'frequency': 35.29}, {...",True
1,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",1,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-0.11186210392338937, 1.664496780604324, 0.5...",None,"[[-0.3727525915918, 1.49596350522284, 0.600103...",-8.672764,-7.991559,"[{'frequency': -42.03}, {'frequency': -37.62},...",True
2,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",2,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-0.003487447036605559, 1.4525343475541515, -...",None,"[[0.3003496445252, 1.49219880823439, -0.427892...",-8.672996,-7.990435,"[{'frequency': 14.28}, {'frequency': 30.36}, {...",True
3,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",3,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-1.0196810753309713, 0.8421778342680636, -0....",None,"[[-1.39868127835716, 0.64856952558997, -0.6009...",-8.679348,-7.998356,"[{'frequency': -29.13}, {'frequency': -7.06}, ...",True
4,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",4,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.5812412715997919, -1.4168195744206293, -0....",None,"[[0.7466897230968, -1.21804805317634, -0.51275...",-8.678083,-7.997728,"[{'frequency': -12.81}, {'frequency': 17.35}, ...",True
5,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",0,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.4390960857886995, -1.6328418133528142, -0....",None,"[[0.43019911652388, -1.72946999811479, -0.3617...",-8.652989,-7.971782,"[{'frequency': -27.34}, {'frequency': -20.92},...",True
6,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",1,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.9005569322225522, -1.571171609182764, 0.38...",None,"[[1.03852390352251, -1.5569221762245, 0.462755...",-8.652888,-7.973079,"[{'frequency': -27.44}, {'frequency': 19.01}, ...",True
7,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",2,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-1.130423699487623, -0.08462180729002457, 1....",None,"[[-1.52747225586051, 0.01026725046857, 1.22386...",-8.642448,-7.959475,"[{'frequency': 26.79}, {'frequency': 38.45}, {...",True
8,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",3,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-1.3521654308051776, 1.1613032005858053, -0....",None,"[[-1.35968846378798, 1.12126146567535, -0.2484...",-8.649504,-7.969544,"[{'frequency': 17.46}, {'frequency': 35.29}, {...",True
9,TS(methyl_furan-2-carboxylate_rpos(4)),methyl_furan-2-carboxylate,4,"[10, 11, 39, 40, 41, 46]",0,COC(C1=CC=CO1)=O,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-0.2293595587104927, -1.6934911667823793, -0...",None,"[[-0.48361369108058, -1.83594650873286, -0.464...",-9.045633,-8.365566,"[{'frequency': 10.26}, {'frequency': 24.42}, {...",True


In [35]:
df_sqmorca = step.orca(
    df=df,
    options={
        "XTB2":    None,
    },
    xtra_inp_str="",    # no extra input
    constraint=False    # no distance constraints
)

[orca-XTB2] row 0 (TS(1-methylpyrrole_rpos(2)))…
[orca-XTB2] row 1 (TS(1-methylpyrrole_rpos(2)))…
[orca-XTB2] row 2 (TS(1-methylpyrrole_rpos(2)))…
[orca-XTB2] row 3 (TS(1-methylpyrrole_rpos(2)))…
[orca-XTB2] row 4 (TS(1-methylpyrrole_rpos(2)))…
[orca-XTB2] row 5 (TS(1-methylpyrrole_rpos(3)))…
[orca-XTB2] row 6 (TS(1-methylpyrrole_rpos(3)))…
[orca-XTB2] row 7 (TS(1-methylpyrrole_rpos(3)))…
[orca-XTB2] row 8 (TS(1-methylpyrrole_rpos(3)))…
[orca-XTB2] row 9 (TS(methyl_furan-2-carboxylate_rpos(4)))…
[orca-XTB2] row 10 (TS(methyl_furan-2-carboxylate_rpos(4)))…
[orca-XTB2] row 11 (TS(methyl_furan-2-carboxylate_rpos(4)))…
[orca-XTB2] row 12 (TS(methyl_furan-2-carboxylate_rpos(4)))…
[orca-XTB2] row 13 (TS(methyl_furan-2-carboxylate_rpos(4)))…
[orca-XTB2] row 14 (TS(methyl_furan-2-carboxylate_rpos(5)))…
[orca-XTB2] row 15 (TS(methyl_furan-2-carboxylate_rpos(5)))…
[orca-XTB2] row 16 (TS(methyl_furan-2-carboxylate_rpos(5)))…
[orca-XTB2] row 17 (TS(methyl_furan-2-carboxylate_rpos(5)))…
[orca-XTB2]

Orca calculation did not terminate normally.


[orca-XTB2] row 24 (TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(8)))…
[orca-XTB2] row 25 (TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(8)))…
[orca-XTB2] row 26 (TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(8)))…
[orca-XTB2] row 27 (TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(8)))…
[orca-XTB2] row 28 (TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(10)))…
[orca-XTB2] row 29 (TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(10)))…
[orca-XTB2] row 30 (TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(10)))…
[orca-XTB2] row 31 (TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(10)))…
[orca-XTB2] row 32 (TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(10)))…
[orca-XTB2] row 33 (TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(11)))…
[orca-XTB2] row 34 (TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(11)))…
[orca-XTB2] row 35 (TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(11)))…
[orca-XTB2] row 36 (TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(11)))…
[orca-XTB2] row 37 (TS(N,N-diethyl-1H-indole-2-carboxamide_rpos(11)))…
[orca-XTB2

ValueError: Length of values (47) does not match length of index (48)

In [ ]:
df_sqmorca

,custom_name,ligand_name,rpos,constraint_atoms,cid,smiles,atoms,coords_embedded,energy_uff,xtb-gfnff-ohess-opt_coords,xtb-gfnff-ohess-electronic_energy,xtb-gfnff-ohess-gibbs_energy,xtb-gfnff-ohess-vibs,xtb-gfnff-ohess-normal_termination,orca-XTB2-electronic_energy,orca-XTB2-normal_termination
0,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",0,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.2239454721163768, -1.5689961861050987, -0....",None,"[[0.30109937249364, -1.64995020686268, -0.6682...",-8.678725,-7.997619,"[{'frequency': 16.89}, {'frequency': 35.29}, {...",True,-65.197821,True
1,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",1,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-0.11186210392338937, 1.664496780604324, 0.5...",None,"[[-0.3727525915918, 1.49596350522284, 0.600103...",-8.672764,-7.991559,"[{'frequency': -42.03}, {'frequency': -37.62},...",True,-65.193176,True
2,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",2,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-0.003487447036605559, 1.4525343475541515, -...",None,"[[0.3003496445252, 1.49219880823439, -0.427892...",-8.672996,-7.990435,"[{'frequency': 14.28}, {'frequency': 30.36}, {...",True,-65.198666,True
3,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",3,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-1.0196810753309713, 0.8421778342680636, -0....",None,"[[-1.39868127835716, 0.64856952558997, -0.6009...",-8.679348,-7.998356,"[{'frequency': -29.13}, {'frequency': -7.06}, ...",True,-65.187357,True
4,TS(1-methylpyrrole_rpos(2)),1-methylpyrrole,2,"[10, 11, 39, 40, 41, 44]",4,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.5812412715997919, -1.4168195744206293, -0....",None,"[[0.7466897230968, -1.21804805317634, -0.51275...",-8.678083,-7.997728,"[{'frequency': -12.81}, {'frequency': 17.35}, ...",True,-65.187839,True
5,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",0,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.4390960857886995, -1.6328418133528142, -0....",None,"[[0.43019911652388, -1.72946999811479, -0.3617...",-8.652989,-7.971782,"[{'frequency': -27.34}, {'frequency': -20.92},...",True,-65.206309,True
6,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",1,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(0.9005569322225522, -1.571171609182764, 0.38...",None,"[[1.03852390352251, -1.5569221762245, 0.462755...",-8.652888,-7.973079,"[{'frequency': -27.44}, {'frequency': 19.01}, ...",True,-65.206017,True
7,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",2,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-1.130423699487623, -0.08462180729002457, 1....",None,"[[-1.52747225586051, 0.01026725046857, 1.22386...",-8.642448,-7.959475,"[{'frequency': 26.79}, {'frequency': 38.45}, {...",True,-65.220614,True
8,TS(1-methylpyrrole_rpos(3)),1-methylpyrrole,3,"[10, 11, 39, 40, 41, 45]",3,CN1C=CC=C1,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-1.3521654308051776, 1.1613032005858053, -0....",None,"[[-1.35973708574836, 1.12110490023986, -0.2488...",-8.649503,-7.969514,"[{'frequency': 17.65}, {'frequency': 35.83}, {...",True,-65.215589,True
9,TS(methyl_furan-2-carboxylate_rpos(4)),methyl_furan-2-carboxylate,4,"[10, 11, 39, 40, 41, 46]",0,COC(C1=CC=CO1)=O,"[C, C, C, C, C, C, H, H, H, H, B, N, C, C, C, ...","[(-0.2293595587104927, -1.6934911667823793, -0...",None,"[[-0.48361369108058, -1.83594650873286, -0.464...",-9.045633,-8.365566,"[{'frequency': 10.26}, {'frequency': 24.42}, {...",True,-76.084564,True


In [ ]:
df_sqmorca['xtb-gfnff-ohess-vibs']

0     [{'frequency': 16.89}, {'frequency': 35.29}, {...
1     [{'frequency': -42.03}, {'frequency': -37.62},...
2     [{'frequency': 14.28}, {'frequency': 30.36}, {...
3     [{'frequency': -29.13}, {'frequency': -7.06}, ...
4     [{'frequency': -12.81}, {'frequency': 17.35}, ...
5     [{'frequency': -27.34}, {'frequency': -20.92},...
6     [{'frequency': -27.44}, {'frequency': 19.01}, ...
7     [{'frequency': 26.79}, {'frequency': 38.45}, {...
8     [{'frequency': 17.65}, {'frequency': 35.83}, {...
9     [{'frequency': 10.26}, {'frequency': 24.42}, {...
10    [{'frequency': -23.4}, {'frequency': 23.14}, {...
11    [{'frequency': -18.66}, {'frequency': -6.86}, ...
12    [{'frequency': 16.69}, {'frequency': 21.38}, {...
13    [{'frequency': 10.21}, {'frequency': 11.77}, {...
14    [{'frequency': 8.75}, {'frequency': 15.44}, {'...
15    [{'frequency': -22.54}, {'frequency': 15.09}, ...
16    [{'frequency': -23.59}, {'frequency': 23.73}, ...
17    [{'frequency': 15.27}, {'frequency': 22.08